# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.


## 参数管理

当我们选定架构并设置超参数后，就进入训练循环。我们的目标是找到能最小化损失函数的参数值。训练完成后，我们需要这些参数来进行未来的预测。此外，有时我们希望：
* 提取参数（可能为了在其他上下文中复用）
* 将模型保存到磁盘以便在其他软件中执行
* 通过检查参数来获得科学理解

大多数情况下，我们可以忽略参数声明和操作的具体细节，依赖深度学习框架来完成繁重工作。然而，当我们需要使用非标准层的定制架构时，有时必须深入处理参数声明和操作的底层细节。本节将涵盖：

* 访问参数（用于调试、诊断和可视化）
* 跨不同模型组件共享参数

In [1]:
import torch
from torch import nn

(**We start by focusing on an MLP with one hidden layer.**)


In [2]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.


## [**参数访问**]
:label:`subsec_param-access`

让我们从如何从已有模型中访问参数开始。

When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.


当通过`Sequential`类定义模型时，
我们可以像处理列表一样通过索引访问任意层，
每个层的参数都储存在其对应的属性中。

We can inspect the parameters of the second fully connected layer as follows.


我们可以通过如下方式查看第二个全连接层的参数：

In [6]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.0731,  0.1670, -0.2303,  0.2594, -0.0652, -0.2690, -0.0368, -0.2228]])),
             ('bias', tensor([-0.1794]))])

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.


可以看到这个全连接层包含两个参数，
分别对应该层的权重和偏置。

### [**目标参数**]

需要注意的是，每个参数都被表示为参数类的实例。
要对参数执行任何操作，首先需要访问底层的数值。
有几种方法可以做到这一点，有些比较简单，而有些则更通用。
下面的代码从第二个神经网络层提取偏置，
返回的是参数类实例，并进一步访问该参数的值。

In [7]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.1794]))

Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.


参数是复杂的对象，包含了数值、梯度以及额外信息。
这就是为什么我们需要显式请求其数值的原因。

除了数值外，每个参数还允许我们访问梯度。
由于我们尚未对此网络调用反向传播，因此梯度处于初始状态。

In [8]:
net[2].weight.grad == None

True

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.


### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们会变得很麻烦。
当处理更复杂的模块（例如嵌套模块）时，
情况会变得特别麻烦，
因为我们需要递归遍历整个模块树来提取每个子模块的参数。
下面的代码演示如何访问所有层的参数：

通过`parameters`方法可以访问所有参数：
```

In [10]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.


## [**参数绑定**]

在多个层之间共享参数通常很有用。
让我们看看如何优雅地实现此操作。
下面我们定义一个全连接层，
然后将其参数显式设置为另一个层的参数。
在访问参数之前，我们需要先运行前向传播
`net(X)`。

In [7]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.


这个例子表明第二层和第三层的参数是绑定的。
它们不仅相等，而且由同一个张量表示。
因此，如果我们改变其中一个参数，
另一个也会随之改变。

You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.


读者可能会疑惑，
当参数被绑定时，
梯度会发生什么变化？
由于模型参数包含梯度，
在反向传播过程中，
第二个隐藏层和第三个隐藏层的梯度会被加在一起。

## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


[Discussions](https://discuss.d2l.ai/t/57)


## 小结

我们有多重方法可以访问和绑定模型参数。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的 `NestMLP` 模型，并访问各个层的参数。
1. 构造一个包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？